<a href="https://colab.research.google.com/github/Rocco000/OncoVision/blob/main/Scripts/Explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ",device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #Connect to Google Drive

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build

#To authenticate the user that run the script in order to use the correct path
auth.authenticate_user()
drive_service = build('drive', 'v3')

#Get user information
about = drive_service.about().get(fields='user').execute()
user_email = about['user']['emailAddress']
script_owner = False

if user_email =="rocco.iul2000@gmail.com":
  script_owner = True
  #Run the .ipynb file
  %run '/content/drive/MyDrive/Colab Notebooks/DatasetLoader.ipynb'
  %run '/content/drive/MyDrive/Colab Notebooks/ModelArchitecture1.ipynb'
  %run '/content/drive/MyDrive/Colab Notebooks/TrainModel.ipynb'
else:
  %run '/content/drive/MyDrive/LinkToOncoVision/DatasetLoader.ipynb'
  %run '/content/drive/MyDrive/LinkToOncoVision/ModelArchitecture1.ipynb'
  %run '/content/drive/MyDrive/LinkToOncoVision/TrainModel.ipynb'

In [ ]:
#Train model
model_configuration,acc,pre,rec,f1=start_process(False,0.001,64,64,1)

In [ ]:
!pip install grad-cam
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F


model = ConvModel1()
model.load_state_dict(model_configuration) #Load the best model configuration
model.to(device)
#Set the model in evaluation mode
model.eval()
target_layers=[model.layer4]
for images, labels in test_loader:
  for image in images:
    input_tensor =image
    break
  for label in labels:
    input_label=label
    break
  break

#Construct the CAM object
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True if device=="cuda" else False)

input_tensor = input_tensor.type(torch.cuda.FloatTensor)
input_tensor=torch.unsqueeze(input_tensor,0)
input_tensor.to(device)
predicted_label=model(input_tensor)
#Apply the Softmax activation function. Dim=1 because the output size is [64,2] where the model prediction is in the second column
prediction = F.softmax(predicted_label, dim=1)

#To extract the predicted class with the highest probability for each input sample. 1 to indicate on which dimension apply the max
_, prediction = torch.max(prediction, 1)


targets = [ClassifierOutputTarget(1)]


grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
grayscale_cam = grayscale_cam[0, :]
#Cooy the tensor in CPU
cpu_tensor=input_tensor.cpu()
#Convert the tensor in the original image
rgb_img= cpu_tensor.numpy()[0]
visualization = show_cam_on_image(rgb_img.transpose(1,2,0), grayscale_cam, use_rgb=False)
print("Effective label:",input_label.numpy())
print("Predicted label:", prediction.cpu().numpy())
plt.imshow(visualization)
plt.axis('off')  # Rimuovi gli assi
plt.show()
